## Try out a simple optimisation routine with scipy's minimize function
Using the same model as in the previous notebook.
$\\i_t = R_t\sum_{\tau<t} i_\tau g_{t-\tau}$

In [ ]:
from typing import Dict
from scipy.stats import gamma
import numpy as np
import pandas as pd
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import minimize

In [ ]:
def get_gamma_params_from_mean_sd(req_mean: float, req_sd: float) -> Dict[str, float]:
    var = req_sd ** 2.0
    scale = var / req_mean
    a = req_mean / scale
    return {'a': a, 'scale': scale}

In [ ]:
# Model parameters
seed = 1.0
gen_time_sd = 1.5
gen_time_mean = 5.0

In [ ]:
# Generation time
times = 40
gamma_params = get_gamma_params_from_mean_sd(gen_time_mean, gen_time_sd)
gen_time_densities = np.diff(gamma.cdf(range(times + 1), **gamma_params))

In [ ]:
dummy_data = pd.Series(
    {
        5: 1.0,
        10: 1.0,
        15: 1.5,
        25: 4.2,
        30: 3.8,
        35: 2.1,
    },
)

In [ ]:
def model_func(process_req):
    process_times = np.linspace(0.0, times, len(process_req))
    process_vals = np.interp(range(times), process_times, process_req)
    incidence = np.zeros(times)
    incidence[0] = seed
    pop = 100.0
    suscept = pop - seed
    suscept_dict = {}
    r_t = {}
    for t in range(1, times):
        suscept_prop = suscept / pop
        infect_contribution_by_day = incidence[:t] * gen_time_densities[t-1::-1]
        this_inc = infect_contribution_by_day.sum() * suscept_prop * process_vals[t]
        r_t[t] = gen_time_densities[t-1::-1].sum() * suscept_prop * process_vals[t]
        incidence[t] = this_inc
        suscept = max(suscept - this_inc, 0.0)
        suscept_dict[t] = suscept
    incidence_df = pd.Series(incidence, index=range(times))
    return incidence_df, pd.Series(suscept_dict), pd.Series(r_t)

def calib_func(process_req):    
    incidence_df, _, _ = model_func(process_req)
    return sum([(incidence_df[t] - d) ** 2 for t, d in dummy_data.items()])

In [ ]:
result = minimize(calib_func, [1.0] * 4, method='Nelder-Mead')
process_times = np.linspace(0.0, times, len(result.x))
process_df = pd.Series(result.x, index=process_times)
optimised_df, suscept_df, r_t = model_func(result.x)

In [ ]:
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(go.Scatter(x=dummy_data.index, y=dummy_data, mode='markers', name='targets'))
fig.add_trace(go.Scatter(x=optimised_df.index, y=optimised_df, name='model'))
fig.add_trace(go.Scatter(x=process_df.index, y=process_df, name='transmission potential'))
fig.add_trace(go.Scatter(x=suscept_df.index, y=suscept_df, name='susceptibles'), secondary_y=True)
fig.add_trace(go.Scatter(x=r_t.index, y=r_t, name='Rt'))